In [ ]:
!wget -q https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/db_loader.py
!wget -q https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/llm_connector.py
!wget -q https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/requirements.txt


!wget -q https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/Files/Demand%20Forecasting%20%26%20Rebalancing%20Planner.pdf
!wget -q https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/Files/Network%20Expansion%20%26%20Capacity%20Sizing.pdf
!wget -q https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/Files/Station%20Availability%20%26%20SLA%20Monitoring.pdf
!wget -q https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/Files/Weather-Aware%20Rider%20Incentives.pdf

# Sample PRDs

1. **Station Availability & SLA Monitoring**
    1. **Purpose:** Detect and prevent “empty/full” station outages with live alerts, on-call rotations, and an ops dashboard.
    2. **Key inputs:** `status` (bikes/docks by timestamp), `station` (location, capacity).**Primary KPIs:** % time stations are in-service (not empty/full), mean time to recover (MTTR), alerts per 10k trips.
2. **Demand Forecasting & Rebalancing Planner**
    1. **Purpose:** Forecast hourly net inflow/outflow per station and generate shift plans/van routes to keep supply balanced.
    2. **Key inputs:** `trip` (start/end, duration), `status` (historical fill curves), `weather` (temp, rain, wind), `station` (dock_count, lat/long).
    3. **Primary KPIs:** Stockout minutes ↓, rebalancing cost/ride ↓, forecast MAE per station-hour.
3. **Weather-Aware Rider Incentives (Crowd Rebalancing)**
    1. **Purpose:** Issue targeted credits to riders to pick up/return at stressed stations when forecasted load or live status crosses thresholds.
    2. **Key inputs:** `status` (live stress), `trip` (origin/destination patterns), `weather` (event-sensitive demand), `station` (capacity, proximity graph).
    3. **Primary KPIs:** % stress events resolved via incentives, incentive cost per resolved event, availability uplift vs. control.
4. **Network Expansion & Capacity Sizing**
    1. **Purpose:** Identify where to add docks/new stations by modeling unmet demand, churn from outages, and micro-mobility catchment gaps.
    2. **Key inputs:** `trip` (OD flows, overflow durations), `status` (chronic empty/full streaks), `station` (geo features), `weather` (seasonality).
    3. **Primary KPIs:** Incremental rides at proposed sites, reduction in nearby stockouts, ROI (rides/$ of infrastructure).

# PRDs:

[Station Availability & SLA Monitoring](https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/Files/Station%20Availability%20%26%20SLA%20Monitoring.pdf)

[Demand Forecasting & Rebalancing Planner](https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/Files/Demand%20Forecasting%20%26%20Rebalancing%20Planner.pdf)

[Weather-Aware Rider Incentives (Crowd Rebalancing)](https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/Files/Weather-Aware%20Rider%20Incentives.pdf)

[Network Expansion & Capacity Sizing](https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/Files/Network%20Expansion%20%26%20Capacity%20Sizing.pdf)

In [ ]:
%pip install -r requirements.txt

# PRD Writer Agent Workshop

Welcome! In this workshop notebook we turn LangChain, Groq, and a Citi Bike SQLite dataset into a Product Requirements Document (PRD) co-pilot. The agent blends structured ride data with curated reference docs so product teams can articulate a commuter-pass concept—without leaving the notebook.

You will guide learners through each layer of the stack: environment preparation, safe SQL tooling, document retrieval, prompting, and live execution.


## Why we rely on Markdown for model context

> Markdown sits closest to the code, version controls cleanly, and renders everywhere our learners will review the notebook.

| Context method | Benefits | Drawbacks |
| --- | --- | --- |
| Markdown cells (this notebook) | Sits inline with execution order; easy to diff and iterate; visible during live teaching. | Requires deliberate authoring effort. |
| Inline code comments | Great for pointing at specific lines, but disappear during demo runs and clutter logic. | Hard for non-coders to follow; limited formatting. |
| External docs / slides | Rich visuals, but easily fall out of sync with the live notebook. | Context switching interrupts the teaching flow. |

Markdown wins because it keeps narrative, prompts, and outputs co-located. Learners can rerun the notebook later and still have the story stitched right next to the cells the model reads.


## Environment setup & context anchors

The next cell imports every library our agent touches—from standard utilities and pandas to LangChain prompt builders. Call out that keeping imports explicit helps models (and humans) understand the available tools, which is essential when you later ask ChatGPT to extend the notebook.


In [ ]:
# Optional: install dependencies (uncomment if needed)

import os, re, glob, json, time, sqlite3, textwrap
from typing import Optional, List, Dict, Any, Literal
from IPython.display import Markdown, display


from dotenv import load_dotenv

# LangChain / Groq
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

# markitdown converts many file formats into Markdown
from markitdown import MarkItDown


## Configure the Groq LLM for PRD drafting

We obtain a Groq-hosted `moonshotai/kimi-k2-instruct-0905` model through a helper. The `.bind(...)` call is where we could control sampling behavior:
- `top_p=1` (if set) keeps the nucleus wide; lowering it or explicitly setting `top_k` would make the model hug the most likely continuations.
- `temperature` is omitted here, so the connector default applies. Encourage learners to experiment with lower values (~0.2) when they want reproducible PRD sections.
- `max_completion_tokens` bounds response length so the agent cannot ramble indefinitely.

Even when some knobs are commented out, pointing to them during the workshop shows participants which levers they can pull when they need more determinism or richer reasoning effort.


In [ ]:
from llm_connector import langchain_groq_llm_connector

llm = langchain_groq_llm_connector("Insert API Key Here","openai/gpt-oss-20b")
llm = llm.bind(
    tools=[{"type":"browser_search"},{"type":"code_interpreter"}],
    compound_custom = {"tools":{"enabled_tools":["web_search","code_interpreter"]}},
    tool_choice="auto",
    reasoning_effort="medium",
    top_p=1,
    max_completion_tokens=8192,
)


# ---- Safety knobs for SQL and results ----------------------------------------
SQL_TIMEOUT_SECONDS = 5000      # wall clock per query
SQL_MAX_RETURN_ROWS = 10000000   # cap result rows to avoid huge pulls

## Link the SQL knowledge base

This cell anchors the Retrieval-Augmented Generation (RAG) pipeline to a consistent data source. We reuse the Citi Bike SQLite path so the agent can fact-check PRD claims—like subscriber distribution or commuter trip volumes—directly from historical rides.


In [ ]:
from db_loader import prepare_citibike_database

DB_PATH, conn, run_query = prepare_citibike_database()
SQLITE_URI = f"sqlite:///{DB_PATH}"

## Guarded SQL tool overview

Before letting the agent touch data, we wrap SQLite access in helper functions that enforce read-only behavior, inject `LIMIT` clauses, and store results on disk. Emphasize that:
- `_is_read_only` and `_add_limit_if_missing` are defensive coding patterns students should reuse when exposing databases to LLMs.
- The `@tool`-decorated `run_sqlite_query` returns JSON metadata (paths, preview Markdown, executed SQL) so downstream tools—and the agent—can chain work predictably.

This design keeps the notebook safe for live demos, even when learners craft creative prompts.


In [ ]:

def _is_read_only(sql: str) -> bool:
    """Allow only SELECT and PRAGMA table_info (read-only)."""
    forbidden = r"\b(INSERT|UPDATE|DELETE|DROP|ALTER|CREATE|REPLACE|TRUNCATE|ATTACH|DETACH|VACUUM|BEGIN|COMMIT)\b"
    if re.search(forbidden, sql, flags=re.IGNORECASE):
        return False
    if re.search(r"\bPRAGMA\b", sql, re.IGNORECASE) and "table_info" not in sql.lower():
        return False
    return True

def _add_limit_if_missing(sql: str, limit: int) -> str:
    """Inject a LIMIT if the user didn't specify one on SELECT queries."""
    q = sql.strip().rstrip(";")
    if not q.lower().startswith("select"):
        return q + ";"
    if re.search(r"\blimit\b", q, flags=re.IGNORECASE):
        return q + ";"
    return f"SELECT * FROM (\n{q}\n) LIMIT {limit};"

@tool("run_sqlite_query", return_direct=False)
def run_sqlite_query(query: str, limit: int = SQL_MAX_RETURN_ROWS) -> str:
    """
    Run a read-only SQL query against the local SQLite DB.
    Safety: Only SELECT or PRAGMA table_info allowed. LIMIT injected if missing for SELECT.
    Returns a small JSON string with file paths and a preview table.
    """
    if DB_PATH is None or not os.path.exists(DB_PATH):
        return json.dumps({"error": "No SQLite database found. Set DB_FILENAME or place a .sqlite/.db in this folder."})
    if not _is_read_only(query):
        return json.dumps({"error": "Only read-only queries allowed (SELECT/PRAGMA table_info)."})
    if query.strip().lower().startswith("select") and "limit" not in query.lower():
        query = _add_limit_if_missing(query, limit)
    conn = sqlite3.connect(DB_PATH, check_same_thread=False)
    conn.execute(f"PRAGMA busy_timeout = {SQL_TIMEOUT_SECONDS*1000};")
    try:
        import pandas as pd
        df = pd.read_sql_query(query, conn)
    except Exception as e:
        return json.dumps({"error": f"SQLite error: {e}"})
    finally:
        conn.close()
    csv_path = os.path.join("outputs", "sql_result.csv")
    json_path = os.path.join("outputs", "sql_result.json")
    try:
        df.to_csv(csv_path, index=False)
        df.to_json(json_path, orient="records")
    except Exception:
        pass
    try:
        preview_md = df.head(10).to_markdown(index=False) if not df.empty else "(no rows)"
    except Exception:
        preview_md = df.head(10).to_string(index=False) if not df.empty else "(no rows)"
    return json.dumps({
        "rows": int(len(df)),
        "columns": list(map(str, df.columns)),
        "csv_path": csv_path,
        "json_path": json_path,
        "preview": preview_md,
        "sql_used": query,
    })


## Document context builder (markitdown + BM25)

The upcoming cell introduces a complementary tool that turns slides, PDFs, and spreadsheets into retrievable context. Highlight how we:
- Accept both `build` (index documents) and `search` (retrieve top chunks) modes through a typed Pydantic schema.
- Use `markitdown` to convert rich formats into Markdown, keeping structure that the LLM can ingest easily.
- Apply a BM25 retriever so queries surface the most relevant passages for the PRD draft.

Together with the SQL tool, this rounds out both halves of RAG: structured data plus unstructured reference material.


In [ ]:
ALLOWED_EXT = {".pdf", ".ppt", ".pptx", ".xls", ".xlsx", ".png", ".jpg", ".jpeg"}

class DocContextArgs(BaseModel):
    mode: Literal["build", "search"] = Field(..., description="'build' to index a folder; 'search' to retrieve relevant chunks.")
    directory: Optional[str] = Field(None, description="Directory path containing source documents (for build mode).")
    query: Optional[str] = Field(None, description="Search query (for search mode).")
    top_k: int = Field(5, description="Number of chunks to return in search mode.")

_CURRENT_INDEX_NAME: Optional[str] = None
_CURRENT_RETRIEVER: Optional[BM25Retriever] = None
_CURRENT_CHUNKS: List[Document] = []

def _collect_files(root_dir: str) -> List[str]:
    files: List[str] = []
    for base, _, fnames in os.walk(root_dir):
        for f in fnames:
            ext = os.path.splitext(f)[1].lower()
            if ext in ALLOWED_EXT:
                files.append(os.path.join(base, f))
    return files

def _md_from_file(md: MarkItDown, path: str) -> str:
    try:
        result = md.convert(path)
        text = result.text if hasattr(result, "text") else str(result)
        return text or ""
    except Exception as e:
        return f"[markitdown error reading {os.path.basename(path)}: {e}]"

@tool("doc_context", args_schema=DocContextArgs, return_direct=False)
def doc_context(mode: str, directory: Optional[str] = None, query: Optional[str] = None, top_k: int = 5) -> str:
    """
    Document context builder and retriever using markitdown + BM25.
    mode=build: index a folder of PDFs, PPT(X), XLS(X), PNG/JPG into Markdown chunks.
    mode=search: retrieve top-k relevant chunks from the last-built index.
    Returns JSON with index summary or match snippets.
    """
    global _CURRENT_INDEX_NAME, _CURRENT_RETRIEVER, _CURRENT_CHUNKS
    if mode == "build":
        if not directory:
            return json.dumps({"error": "Provide 'directory' for build mode."})
        if not os.path.isdir(directory):
            return json.dumps({"error": f"Directory not found: {directory}"})
        files = _collect_files(directory)
        if not files:
            return json.dumps({"error": f"No supported files found under: {directory}"})
        mkd = MarkItDown()
        docs: List[Document] = []
        for fp in files:
            md_text = _md_from_file(mkd, fp)
            docs.append(Document(page_content=md_text, metadata={"source": fp}))
        splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=150)
        chunks = splitter.split_documents(docs)
        retriever = BM25Retriever.from_documents(chunks)
        retriever.k = 6
        _CURRENT_INDEX_NAME = os.path.abspath(directory)
        _CURRENT_RETRIEVER = retriever
        _CURRENT_CHUNKS = chunks
        summary = {
            "indexed_dir": _CURRENT_INDEX_NAME,
            "file_count": len(files),
            "chunk_count": len(chunks),
            "sample_files": [os.path.basename(p) for p in files[:5]],
        }
        return json.dumps(summary)
    elif mode == "search":
        if _CURRENT_RETRIEVER is None:
            return json.dumps({"error": "No index built yet. Call doc_context with mode='build' first."})
        if not query:
            return json.dumps({"error": "Provide 'query' for search mode."})
        _CURRENT_RETRIEVER.k = max(1, min(int(top_k), 20))
        results = _CURRENT_RETRIEVER.get_relevant_documents(query)
        def to_snippet(d: Document) -> Dict[str, Any]:
            txt = d.page_content.replace("\n", " ")
            snippet = (txt[:400] + "...") if len(txt) > 400 else txt
            return {"source": os.path.basename(d.metadata.get("source", "unknown")), "snippet": snippet}
        out = {
            "indexed_dir": _CURRENT_INDEX_NAME,
            "query": query,
            "top_k": _CURRENT_RETRIEVER.k,
            "matches": [to_snippet(d) for d in results],
        }
        return json.dumps(out)
    else:
        return json.dumps({"error": "Unknown mode. Use 'build' or 'search'."})


### RAG workflow cheat sheet

> Retrieve from SQL **and** documents, then generate the PRD narrative.

1. **Indexing** – Convert docs to Markdown and store BM25 vectors.
2. **Query planning** – Agent decides whether it needs SQL metrics, document insights, or both.
3. **Retrieval** – `run_sqlite_query` returns tables; `doc_context` returns annotated excerpts.
4. **Synthesis** – The LLM weaves retrieved facts into PRD sections.

- [x] Structured data captured (SQLite)
- [x] Unstructured context captured (Docs folder)
- [ ] Manual copy/paste (no longer required!)

| Stage | Tool/Prompt | Output |
| --- | --- | --- |
| 1 | `doc_context` in `build` mode | Indexed Markdown corpus |
| 2 | Agent PLAN (Thought steps) | Tool call decisions |
| 3 | `run_sqlite_query` + `doc_context` (`search`) | JSON payloads w/ data + excerpts |
| 4 | LLM completion under system prompt | Polished PRD Markdown |

```text
[User Brief]
    ↓
(System Prompt + PLAN)
    ↓
[SQL Tool] → metrics.csv/json
[Doc Tool] → supporting excerpts
    ↓
      LLM synthesizes PRD
```

Markdown gives us flexible visuals—lists, tables, ASCII diagrams—that render perfectly in Jupyter and keep the retrieval story close to the code.


## PRD system prompt guidance

This prompt is the agent’s **prefix** message: it persists across turns and overrides conflicting user instructions. Walk learners through best practices evident here:
- Scope the mission (“produce a clear, complete PRD”) and constrain data sources.
- Include explicit refusal guidance to prevent hallucinated facts.
- Call out output formatting expectations (Markdown headings, tables, bullet points) so downstream audiences can read the draft immediately.

Encourage participants to keep system prompts directive, modular (use lists over paragraphs), and transparent about tone and structure. That way, user prompts can stay concise.


In [ ]:
PRD_SYSTEM_PROMPT = textwrap.dedent(
    """
    You are a meticulous Product Requirements Document (PRD) writer.
    Your job is to produce a clear, complete PRD in Markdown using ONLY information derived from:
      - SQL research via the provided SQLite tool, and
      - Document context provided via the document context tool.

    Do not fabricate facts or numbers. If information is unavailable, mark it clearly as a gap.
    Always call tools when you need data or supporting context.
    Reference the documents that you have used in a "Sources" section at the end. Also tag which parts have used which references in the footnote.

    OUTPUT FORMAT (Markdown, in this exact order):
    # <Product Name>
    
    ## Summary
    - One–two sentence overview of the product and its value.
    - Who benefits and in what scenario.
    
    ## Goals
    - Goal 1
    - Goal 2
    
    ## Non-Goals
    - Explicitly list what is out of scope.
    
    ## Background
    - Problem statement and context.
    - Key constraints, assumptions, and dependencies (cite sources).
    
    ## Target Users & Personas
    - Persona(s) with brief needs and pain points.
    
    ## Use Cases
    - Use case 1: short description.
    - Use case 2: short description.
    
    ## Functional Requirements
    | ID | Requirement | Priority | Acceptance Criteria |
    |----|-------------|----------|---------------------|
    | FR-1 | ... | P0/P1/P2 | Concrete, testable criteria |
    | FR-2 | ... | P0/P1/P2 | Concrete, testable criteria |
    
    ## Expected Behavior
    Describe key flows as scenarios in a table.
    | Scenario | Given | When | Then |
    |----------|-------|------|------|
    | Example | Starting context | Action taken | Expected result |
    
    ## Edge Cases & Error Handling
    - How the feature interacts with core functionalities.
    - How the interaction between this feature and other features is handled.

    ## UX & Platform Considerations
    - Platforms, accessibility, localization, performance notes.
    
    ## Data & Analytics
    - Key metrics, events to track, and success criteria.
    - Include any SQL used for metric definitions in a fenced code block.
    
    ## Risks & Open Questions
    - Risk 1 and mitigation.
    - Open question 1.
    
    ## Milestones (Optional)
    - Rough phases and dates if available.
    
    ## Sources
    - List document filenames and/or a short description of SQL datasets used.

    TOOL USAGE GUIDELINES:
    - Use `doc_context` in mode="build" once to index the user-provided folder before drafting.
    - Use `doc_context` in mode="search" whenever you need supporting context.
    - Use `run_sqlite_query` for quantitative data; show SQL in the PRD where relevant.
    - If a tool returns an error or there is insufficient context, state the gap explicitly in the PRD.

    IMPORTANT:
    - Keep responses concise and factual; do not include chain-of-thought.
    - Do not assume file paths or database schema beyond tool results.
    - Always produce valid Markdown matching the structure above.
    """
).strip()


## Agent wiring and control knobs

Here we assemble the LangChain `ChatPromptTemplate`, plug in both tools, and wrap everything with `AgentExecutor`.
- `MessagesPlaceholder("agent_scratchpad")` captures intermediate Thought/Action/Observation traces, making the agent explainable in workshops.
- `max_iterations=8` is the guardrail that complements `top_p`/`temperature`: it caps how many tool calls the agent may attempt before stopping.
- `handle_parsing_errors=True` keeps the session resilient if the LLM emits a slightly malformed JSON chunk.

Link these settings back to earlier discussions about controlling creativity and determinism—prompting is only half the story; runtime limits complete the toolkit.


In [ ]:
PROMPT = ChatPromptTemplate.from_messages([
    ("system", PRD_SYSTEM_PROMPT),
    ("human", "PRD outline: {prd_outline}\n\nIf you need documents, index directory: {docs_dir}\nProceed."),
    MessagesPlaceholder("agent_scratchpad"),
])

TOOLS = [run_sqlite_query, doc_context]

# Try to bind tools to the model for better compatibility; fall back if unavailable
try:
    llm_for_agent = llm.bind_tools(TOOLS)
except Exception:
    llm_for_agent = llm

# Build the tool-calling agent
agent_runnable = create_tool_calling_agent(llm_for_agent, TOOLS, PROMPT)

# Construct executor with version-compatible kwargs
try:
    executor = AgentExecutor(
        agent=agent_runnable,
        tools=TOOLS,
        verbose=True,
        return_intermediate_steps=True,
        max_iterations=20,
        handle_parsing_errors=True,
    )
except TypeError:
    # Fallback for older AgentExecutor that doesn't accept handle_parsing_errors
    executor = AgentExecutor(
        agent=agent_runnable,
        tools=TOOLS,
        verbose=True,
        return_intermediate_steps=True,
        max_iterations=20,
    )

print("PRD Agent ready.")


## Run helper utility

`run_prd_writer` orchestrates a full pass end-to-end: it optionally builds the document index, runs the agent, and captures both the final PRD and intermediate artifacts. Stress how pre-building the index reduces latency during live demos and how saving outputs to `outputs/` gives students artifacts to review after class.


In [ ]:
def run_prd_writer(prd_outline: str, docs_dir: Optional[str] = None) -> Dict[str, Any]:
    # Optionally build the doc index up-front to help the agent
    try:
        if docs_dir and os.path.isdir(docs_dir):
            _ = doc_context.invoke({
                "mode": "build",
                "directory": docs_dir,
                "query": None,
                "top_k": 5,
            })
    except Exception:
        pass
    inputs = {"prd_outline": prd_outline, "docs_dir": docs_dir or "(none provided)"}
    return executor.invoke(inputs)

def save_markdown(md_text: str, out_path: Optional[str] = None) -> str:
    ts = time.strftime("%Y%m%d-%H%M%S")
    final_path = out_path or os.path.join("outputs", f"prd_{ts}.md")
    os.makedirs(os.path.dirname(final_path), exist_ok=True)
    with open(final_path, "w", encoding="utf-8") as f:
        f.write(md_text)
    return final_path


## Example invocation & commuter-pass focus

We seed the agent with a commuter-pass outline and point it at the shared documents directory. During the workshop you can:
- Swap in different outlines (e.g., targeting weekend riders) and compare results.
- Show how retrieved SQL metrics and document excerpts appear in the agent trace.
- Emphasize that the final Markdown can be copied straight into a PRD template or product wiki.


In [ ]:
# Provide your outline and optional docs directory, then run.
prd_outline = "Write a PRD for a commuter pass feature where we offer passes for a certain origin-destation station combination valid for a month in exchange for a flat fee for non-subscribers at a cheaper price compared to the subscription."
docs_dir = "/content"

result = run_prd_writer(prd_outline, docs_dir)
output_text = result.get("output", "") if isinstance(result, dict) else str(result)
print(output_text[:1000] + ("..." if len(output_text) > 1000 else ""))

if output_text:
    path = save_markdown(output_text)
    print(f"Saved PRD to: {path}")


In [ ]:
display(Markdown(output_text))

## Sandbox cell for live exploration

Use the final empty cell to experiment in front of learners—inspect saved Markdown files, tweak prompts, or prototype follow-up utilities—without disrupting the main workflow.
